In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [11]:
#load_summarize_chain 을 이용한 다양한 요약기 구축
# => load_summarize_chain() 함수는 영문으로 요약됨.
#
#요약기를 구축할 때 중심적인 질문은 문서를 LLM의 컨텍스트 창에 어떻게 전달할 것인가입니다. 
#
# 이를 위한 두 가지 일반적인 접근 방식은 다음과 같습니다:
# 1. Stuff: 단순히 모든 문서를 단일 프롬프트로 "넣는" 방식입니다. 이는 가장 간단한 접근 방식입니다.
# 2-1.Map-reduce: 각 문서를 "map" 단계에서 개별적으로 요약한 다음, "reduce" 단계에서 요약본들을 최종 요약본으로 합치는 방식입니다.
# 2-2. Refine: 입력 문서를 순회하며 반복적으로 답변을 업데이트하여 응답을 구성합니다. 각 문서에 대해, 모든 비문서 입력, 현재 문서, 그리고 최신 중간 답변을 LLM chain에 전달하여 새로운 답변을 얻습니다.

import os

from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler

# 웹 기반 문서 로더를 초기화합니다.
#loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# docs = loader.load()

# ./data 폴더에 있는 .txt 확장자 파일을 모두 불러옴.
data_folder = "./data"
docs = []
for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        loader = TextLoader(file_path)
        docs.extend(loader.load())

# 콜백 설정
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(f"{token}", end="", flush=True)


# 모델 설정
# OpenAI의 Chat 모델을 초기화합니다. 
# 여기서는 온도를 0으로 설정하고 모델 이름을 지정합니다.
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[MyCallbackHandler()],
)

# 요약 체인 설정, 
# 여기서 chain_type="stuff" 또는 chain_type="map_reduce" 또는 chain_type="refine"
chain=load_summarize_chain(llm, chain_type="map_reduce")

# 예측
response=chain.invoke({"input_documents": docs})
print(response)
print(f'--'*20)
print(response["output_text"])

This document outlines the regulations for providing support for celebratory events and condolences in the workplace. It specifies the eligibility criteria, the types of events covered, the amount of financial support provided, and the required documentation. It also mentions that if a public holiday coincides with a celebratory event, the holiday will not be counted as a vacation day. Additionally, it states that if an employee gets married within one month of their retirement, they will still be eligible for support. The document also includes some additional guidelines and specifies the effective dates of the regulations.The purpose of this regulation is to establish guidelines for the salary of employees at MocoMsys Co., Ltd. The regulation applies to all employees, except for those specified by other laws or regulations. The salary is composed of basic salary, allowances, bonuses, and retirement benefits. Salary adjustments are typically implemented once a year on January 1st. The

In [12]:
# 방법1) StuffDocumentsChain() 를 이용해서 요약
import os
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain import hub

# 원격 저장소에서 프롬프트를 가져옴.
prompt = hub.pull("teddynote/summary-stuff-documents-korean")

# ./data 폴더에 있는 .txt 확장자 파일을 모두 불러옴.
data_folder = "./data"
docs = []
for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        loader = TextLoader(file_path)
        docs.extend(loader.load())

# 콜백 설정
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(f"{token}", end="", flush=True)

# 모델 설정
# OpenAI의 Chat 모델을 초기화합니다. 
# 여기서는 온도를 0으로 설정하고 모델 이름을 지정합니다.
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[MyCallbackHandler()],
)

# llm 체인 설정
llm_chain=LLMChain(llm=llm, prompt=prompt)

# StuffDocumentsChain 설정
stuff_chain=StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

# 예측
response=stuff_chain.invoke({"input_documents": docs})
print(response["output_text"])


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


📌 경조사 지원 규정:
- 적용대상: 계약직 사원을 포함한 임직원
- 경조사 지원기준: 결혼, 자녀 결혼, 형제자매결혼, 자녀 출산, 부모 회갑 등에 대한 경조금 지급
- 신청서류: 경조금 지급신청서 및 증빙서류 1부
- 경조휴가일수: 공휴일 제외한 일수로 계산
- 결혼퇴직: 퇴직 1개월 이내에 결혼 시 경조금 지급
- 기타: 경조금 신청 시 휴가 신청도 함께 진행해야 함

📌 급여규정:
- 목적: 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행
- 적용범위: 관계법령 및 별도로 정한 사항을 제외하고는 규정에 따름
- 급여의 구성: 기본급, 제수당, 상여금, 퇴직금으로 구분
- 급여조정: 년 1회 1월 1일부로 실시
- 계산과 지급: 매월 1일부터 당월 말일까지 계산, 매월 20일에 지급
- 신규채용 및 복직자 급여: 발령일로부터 일할계산하여 지급
- 휴직자 급여: 휴직기간에 따라 지급
- 퇴직자 급여: 퇴직 전일까지의 급여를 일할계산하여 지급
- 제수당: 필요시 별도로 정하여 지급
- 상여금: 매년 경영성과 등을 고려하여 지급
- 퇴직금: 1년 이상 근속한 직원에게 지급, 근속기간에 따라 산정하여 지급📌 경조사 지원 규정:
- 적용대상: 계약직 사원을 포함한 임직원
- 경조사 지원기준: 결혼, 자녀 결혼, 형제자매결혼, 자녀 출산, 부모 회갑 등에 대한 경조금 지급
- 신청서류: 경조금 지급신청서 및 증빙서류 1부
- 경조휴가일수: 공휴일 제외한 일수로 계산
- 결혼퇴직: 퇴직 1개월 이내에 결혼 시 경조금 지급
- 기타: 경조금 신청 시 휴가 신청도 함께 진행해야 함

📌 급여규정:
- 목적: 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행
- 적용범위: 관계법령 및 별도로 정한 사항을 제외하고는 규정에 따름
- 급여의 구성: 기본급, 제수당, 상여금, 퇴직금으로 구분
- 급여조정: 년 1회 1월 1일부로 실시
- 계산과 지급: 매월 1일부터 당월 말일까지 계산, 매월 20일에 지급
- 신규채용 및 복직자 급여: 발령일로부터 일

In [14]:
# 방법2) Map-Reduce
import os
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain import hub

# langchain 허브에서 'rlm/map-prompt'를 가져옵니다.
map_prompt = hub.pull("teddynote/map-prompt")

# ./data 폴더에 있는 .txt 확장자 파일을 모두 불러옴.
data_folder = "./data"
docs = []
for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        loader = TextLoader(file_path)
        docs.extend(loader.load())

# 콜백 설정
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(f"{token}", end="", flush=True)

# 모델 설정
# OpenAI의 Chat 모델을 초기화합니다. 
# 여기서는 온도를 0으로 설정하고 모델 이름을 지정합니다.
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[MyCallbackHandler()],
)

# llm 체인 설정
map_chain =LLMChain(llm=llm, prompt=map_prompt)

#-----------------------------------------------------------------------
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# reduce 프롬프트 생성
reduce_prompt = hub.pull("teddynote/reduce-prompt-korean")

# 연쇄 체인 생성
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서 리스트를 받아 하나의 문자열로 결합한 후 LLMChain에 전달
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# 매핑된 문서들을 결합하고 반복적으로 축소
reduce_documents_chain = ReduceDocumentsChain(
    # 최종적으로 호출되는 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # `StuffDocumentsChain`의 컨텍스트를 초과하는 문서들을 처리
    collapse_documents_chain=combine_documents_chain,
    # 문서들을 그룹화할 최대 토큰 수.
    token_max=4096,
)
#---------------------------------------------------------------------
# 우리의 map과 reduce 체인을 하나로 결합해 봅시다.
map_reduce_chain = MapReduceDocumentsChain(
    # 매핑 체인
    llm_chain=map_chain,
    # 리듀스 체인
    reduce_documents_chain=reduce_documents_chain,
    # llm_chain에서 문서들을 넣을 변수 이름
    document_variable_name="docs",
    # 매핑 단계의 결과를 출력에 포함시킴
    return_intermediate_steps=False,
)
#--------------------------------------------------------

# 문자를 separators 기준으로 텍스트를 분할하는 객체 생성
# => chunk_size와 오버랩 사이즈를 적용
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

# 문서들을 분할
split_docs = text_splitter.split_documents(docs)
#--------------------------------------------------------

# split_docs를 map_reduce_chain의 run 메서드에 전달하여 실행한 결과를 출력합니다.
summary_result = map_reduce_chain.invoke({"input_documents": split_docs})

print(summary_result)

1. The document outlines the regulations for providing support for family events and milestones within the organization.
2. It specifies the eligibility criteria for employees, including contract workers, and the different types of events covered such as marriages, births, and deaths.
3. The document details the specific support provided for each event, including the number of days off, monetary amounts, and additional benefits like floral arrangements.
4. It also includes information on the required documentation for applying for support, the calculation of leave days, and special provisions for marriage retirement.
5. The regulations emphasize the importance of submitting both support and leave requests together, outline the process for exceptional cases, and specify the effective dates of the regulations and any amendments.1. Purpose of the regulations: The document aims to establish guidelines for the salary of employees at MoCoEmSys in order to implement rational salary management

In [15]:
print(summary_result["output_text"])

1. Regulations for providing support for family events and milestones within the organization.
2. Eligibility criteria for employees, including contract workers, and types of events covered.
3. Specific support provided for each event, including days off, monetary amounts, and additional benefits.
4. Required documentation for applying for support and calculation of leave days.
5. Emphasis on submitting support and leave requests together and process for exceptional cases.
6. Purpose of establishing guidelines for salary management at MoCoEmSys.
7. Composition of salary including basic salary, bonuses, incentives, and retirement benefits.
8. Salary adjustment implemented once a year on January 1st.
9. Payment schedule for salaries and deductions for third-party compensation.
10. Eligibility for retirement benefits, calculation of length of service, and implementation dates of regulations.

요약:
조직 내 가족 행사 및 이벤트에 대한 지원 규정, 직원 자격 기준 및 지원 이벤트 유형, 각 이벤트에 대한 구체적인 지원 내용, 휴가 일수 및 계산, 지원 및 휴가 요